## ① データのアップデート

In [36]:
#ライブラリのインポート

import pandas as pd
import requests
from bs4 import BeautifulSoup

In [37]:
#　requestsメソッドを使って、サイトからHTMLの情報を抽出する。
url = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vTy1ZbSq5OAVbEqScDw_Uf_KfLKydg0uUWBaBEPK_DXaBPCHmwv4ru9MzZHHnVHVmRVvLvOlndJ2ZNZ/pubhtml?gid=159569114&amp;single=true&amp;widget=true&amp;headers=false&amp;gid=0&amp;range=A:F%22'
res = requests.get(url)

In [38]:
#　BeautifulSoup()の設定を行う。
soup = BeautifulSoup(res.text,'html.parser')

In [39]:
# 最新弾の情報を抽出する。
poke_new = soup.find('div',id ='1490875147')
# s3(カード名、型番）の要素を抽出する。
card_data = poke_new.find_all('td')

#情報をリストに格納する。
card_name = []
card_spec = []
card_rare = []
card_num = []
card_type = []
card_price = []

for i in range(0,349):
    card_name.append(card_data[6*i].string)
    card_spec.append(card_data[6*i+1].string)
    card_rare.append(card_data[6*i+2].string)
    card_num.append(card_data[6*i+3].string)
    card_type.append(card_data[6*i+4].string)
    card_price.append(card_data[6*i+5].string)
    
# データの結合を行う。
df_new = pd.concat([
    pd.DataFrame(card_name),
    pd.DataFrame(card_spec),
    pd.DataFrame(card_rare),
    pd.DataFrame(card_num),
    pd.DataFrame(card_type),
    pd.DataFrame(card_price)],axis = 1)
#0行目をカラムに設定する。
df_new.columns = df_new.iloc[0]
df_new = df_new.drop(0)

#価格が欠損値になっている行を削除する。
df_new = df_new.dropna(subset = ['買取'])

#買取データの日付を取得する。
import datetime 
today = datetime.date.today()
today = today.strftime('%y-%m-%d')

df_new.columns = ['カード名', '仕様', 'R', '型番', 'タイプ',today]

In [40]:
#### 通常の情報を抽出する。
poke_norm = soup.find('div',id ='159569114')
# s3(カード名、型番）の要素を抽出する。
card_data = poke_norm.find_all('td')

#情報をリストに格納する。
card_name = []
card_spec = []
card_rare = []
card_num = []
card_type = []
card_price = []

for i in range(0,3644):
    card_name.append(card_data[6*i].string)
    card_spec.append(card_data[6*i+1].string)
    card_rare.append(card_data[6*i+2].string)
    card_num.append(card_data[6*i+3].string)
    card_type.append(card_data[6*i+4].string)
    card_price.append(card_data[6*i+5].string)

# データの結合を行う。
df_norm = pd.concat([
    pd.DataFrame(card_name),
    pd.DataFrame(card_spec),
    pd.DataFrame(card_rare),
    pd.DataFrame(card_num),
    pd.DataFrame(card_type),
    pd.DataFrame(card_price)],axis = 1)
#0行目をカラムに設定する。
df_norm.columns = df_norm.iloc[0]
df_norm = df_norm.drop(0)

#価格が欠損値になっている行を削除する。
df_norm = df_norm.dropna(subset = ['買取価格'])

#買取データの日付を取得する。
import datetime
today = datetime.date.today()
today = today.strftime('%y-%m-%d')

df_norm.columns = ['カード名', '仕様', 'R', '型番', 'タイプ',today]

In [41]:
#更新日のデータを完成させる。
#データを結合する。
df_today = pd.concat([df_norm,df_new],ignore_index = True)
df_today.columns = ['カード名', '仕様', 'R', '型番', 'タイプ',today]

In [42]:
#集計データを読み込む。
df_preday = pd.read_csv('card_data.csv')

In [43]:
#結合のためのkeyを作る。(df_all)

df_today['key'] = df_today['カード名'].str.cat(df_today['型番'],sep = ' , ',na_rep='')
df_today['key'] = df_today['key'].str.cat(df_today['仕様'],sep = ' ,' ,na_rep='')
df_today['key'] = df_today['key'].str.cat(df_today['R'],sep = ' ,' ,na_rep='')

# print('df_todayのユニークなkeyの個数:',len(df_today['key'].unique()))
# print('df_todayのデータの列数:',len(df_today))
# print('df_todayのkeyに重複のあるデータ:\n',df_today[df_today['key'].duplicated()])

In [44]:
#結合のためのkeyを作る。(df_all)

df_preday['key'] = df_preday['カード名'].str.cat(df_preday['型番'],sep = ' , ',na_rep='')
df_preday['key'] = df_preday['key'].str.cat(df_preday['仕様'],sep = ' ,' ,na_rep='')
df_preday['key'] = df_preday['key'].str.cat(df_preday['R'],sep = ' ,' ,na_rep='')

# print('df_predayのユニークなkeyの個数:',len(df_preday['key'].unique()))
# print('df_predayのデータの列数:',len(df_preday))
# print('df_predayのkeyに重複のあるデータ:\n',df_preday[df_preday['key'].duplicated()])

In [45]:
#keyが被る行にtodayの数値を代入する。

df_preday[today] = 0
df_preday

for i in range(len(df_preday)):
    for j in range(len(df_today)):
        if df_preday.loc[i,'key'] == df_today.loc[j,'key']:
            df_preday.loc[i,today] = df_today.loc[j,today]
            break

In [46]:
count = 0
for i in range(len(df_today)):
    flg = 0
    for j in range(len(df_preday)):
        if df_today.loc[i,'key'] == df_preday.loc[j,'key']:
            flg = 1
            break
            
    if flg == 0:
        df_preday = pd.concat([df_preday,df_today[i:i+1]])
        df_preday.reset_index(drop = True,inplace=True)
        count = count + 1

In [47]:
#データの整理を行う。
df_update = df_preday
df_update = df_update.drop('key',axis = 1)

#データの上書きを行う。
df_update.to_csv('card_data.csv',index = None)
df_update.to_csv('/Users/kanta/Desktop/Pokemon_work/data/{}.csv'.format(today),index = None)

## ② データの分析

In [48]:
#数値をint型に変換する。
df_update = df_update.replace(',','', regex=True)

#欠損値の処理を行う。
df_update.fillna({'カード名':'-','仕様':'-','型番':'-','タイプ':'-'},inplace = True)
df_update.fillna(0,inplace = True)
df_update = pd.concat([df_update.iloc[:,:4],df_update.iloc[:,5:].astype(int)],axis = 1)

#前日との値差を計算する。
from datetime import date, timedelta
today = datetime.date.today()
yesterday = today - timedelta(days = 1) 

today = today.strftime('%y-%m-%d')
yesterday = yesterday.strftime('%y-%m-%d')

df_update['前日との値差'] = df_update[today] - df_update[yesterday]  

In [49]:
#値上がりしたカードを抽出する。
print('■ 前日からの値上がりしたカード')
up_num = len(df_update[df_update['前日との値差'] > 0])
pd.set_option('display.max_rows',up_num)
df_update.sort_values('前日との値差',ascending=False)[0:up_num]

■ 前日からの値上がりしたカード


,カード名,仕様,R,型番,21-01-25,21-01-26,21-01-27,21-01-28,21-01-29,前日との値差
1845,ピカチュウ,-,P,279/XY-P,150000,150000,150000,150000,180000,30000
3690,ピカチュウGX,封筒付き,P,393/SM-P,0,0,0,0,11000,11000
2967,ルチア,-,SR,104/096,16000,16000,16000,16000,20000,4000
1873,ピカチュウ,チャンピオンズリーグ2017,P,068/SM-P,18000,18000,18000,18000,20000,2000
1504,カスミのやる気,-,SR,099/087,15000,15000,15000,15000,17000,2000
1017,マリィ,-,SR,198/190,22000,22000,22000,24000,25000,1000
52,ファイヤー＆サンダー＆フリーザーGX,-,UR,226/173,3800,5000,5500,6000,7000,1000
42,レシラム＆リザードンGX,-,UR,220/173,20000,25000,25000,25000,26000,1000
2443,ジラーチ,-,P,322/SM-P,2000,2000,2000,2000,2500,500
1337,ピッピ,-,P,381/SM-P,5500,6500,7000,7000,7500,500


In [50]:
print('■ 前日からの値下がりが大きいカード')
down_num = len(df_update[df_update['前日との値差'] < 0])
pd.set_option('display.max_rows',down_num)
df_update.sort_values('前日との値差',ascending=True).head(down_num)

■ 前日からの値下がりが大きいカード


,カード名,仕様,R,型番,21-01-25,21-01-26,21-01-27,21-01-28,21-01-29,前日との値差
3538,ミュウツーEX,フォトンウェーブ,UR,065/059,3500,3500,3500,3500,0,-3500
3537,ミュウツーEX,バーストボール,UR,065/059,3500,3500,3500,3500,0,-3500
61,リザードンVMAX,-,SSR,308/190,21000,21000,21000,21000,19000,-2000
60,リザードンV,-,SSR,307/190,11000,11000,11000,11000,10000,-1000
683,回収ネット,-,UR,114/096,4200,4200,4200,4200,4000,-200
1482,リーリエのピッピ人形,-,UR,074/049,1700,1700,1700,1700,1500,-200
3536,ミュウツーEX,-,-,014/046,100,100,100,100,0,-100
3535,ミュウツーEX,-,R,028/052,100,100,100,100,0,-100
104,アローラキュウコンGX,-,SSR,213/150,600,600,600,600,500,-100
3534,ミュウツーEX,-,-,050/131,30,30,30,30,0,-30


In [51]:
#新規に値段がついたカードを表示する。
print('■ 新規にデータを追記したカード')
df_update.tail(count)

■ 新規にデータを追記したカード


,カード名,仕様,R,型番,21-01-25,21-01-26,21-01-27,21-01-28,21-01-29,前日との値差
3686,ガラルバリコオル,シャッフルダンス,U,019/070,0,0,0,0,10,10
3687,ガラルバリコオル,-,-,030/190,0,0,0,0,10,10
3688,ポットデス,-,U,037/070,0,0,0,0,10,10
3689,ポットデス,-,-,077/190,0,0,0,0,10,10
3690,ピカチュウGX,封筒付き,P,393/SM-P,0,0,0,0,11000,11000


In [52]:
pd.set_option('display.max_rows',100)
df_update[df_update['カード名'].str.contains('シャワーズ')]

,カード名,仕様,R,型番,21-01-25,21-01-26,21-01-27,21-01-28,21-01-29,前日との値差
426,シャワーズ,-,P,071/S-P,100,100,100,100,100,0
1399,シャワーズ,-,-,275/SM-P,150,150,150,150,150,0
1400,シャワーズ,-,-,033/173,10,10,10,10,10,0
1401,シャワーズ,ミラー,-,033/173,50,50,100,100,100,0
1402,シャワーズGX,RR仕様,-,007/038,150,150,150,150,150,0
3189,シャワーズ LV.42,-,R,旧裏,100,100,100,100,100,0
3355,シャワーズ☆,未開封,-,022/PLAY,65000,65000,65000,65000,65000,0
3356,シャワーズ☆,-,-,022/PLAY,45000,45000,45000,45000,45000,0
3357,シャワーズ☆,1ED,☆,015/108,35000,35000,35000,35000,35000,0
3358,シャワーズ☆,-,☆,015/108,25000,25000,25000,25000,25000,0
